In [1]:
from binance.um_futures import UMFutures
import plotly.graph_objects as go
import requests
import json
import numpy as np
import pandas as pd
import strats
import vectorbt as vbt
from pandas import Timestamp
from sklearn.linear_model import LinearRegression
import datetime
from chart import Chart
from data_ops import DataGetter
data_getter=DataGetter("iPQMe46exUy10KiBaBahQT7ow1uzb9jaxlKj19Bg5BI8JEwJL5bw9LCvJtfKuVbP","J4FGJQdGPpNGm8jpnj3IuFuDJuAe7FIa3cM51L4z662UYux3qD0ByLJ0bfIltZvK")

In [2]:
current_capital = 44.38

In [3]:
data = data_getter.get_pair_data(pair_in="BTCUSDT",interval_in="15m")

In [4]:
chz_chart = Chart(data)

In [5]:
chz_chart.draw_ema(data,order= 100)

In [6]:
chz_chart.draw_donchian(data,14)

In [7]:
chz_chart.plot()

In [8]:
signals = strats.donchian_strat(data)
signals

,Long Signal,Short Signal,Exit Signal
,,,
2024-03-07 22:00:00,False,False,False
2024-03-07 22:15:00,False,False,False
2024-03-07 22:30:00,False,False,False
2024-03-07 22:45:00,False,False,False
2024-03-07 23:00:00,False,False,False
...,...,...,...
2024-03-13 01:45:00,False,False,False
2024-03-13 02:00:00,False,False,False
2024-03-13 02:15:00,False,False,False


In [10]:
Long_PNL = vbt.Portfolio.from_signals(data["Close"], signals["Long Signal"], signals["Exit Signal"], init_cash=100)
Short_PNL = vbt.Portfolio.from_signals(data["Close"], signals["Short Signal"], signals["Exit Signal"], init_cash=100)

In [11]:
pnl = Long_PNL.total_profit() + Short_PNL.total_profit()

In [12]:
pnl

8.17954839528754

In [14]:
#Long_PNL.value().vbt.plot() portfolio plotting function

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': 'ce109782-d6f1-43d3-9ddc-167f60ac2587',
              'x': array([datetime.datetime(2024, 3, 7, 22, 0),
                          datetime.datetime(2024, 3, 7, 22, 15),
                          datetime.datetime(2024, 3, 7, 22, 30), ...,
                          datetime.datetime(2024, 3, 13, 2, 15),
                          datetime.datetime(2024, 3, 13, 2, 30),
                          datetime.datetime(2024, 3, 13, 2, 45)], dtype=object),
              'y': array([100.        , 100.        , 100.        , ..., 106.92169364,
                          106.94264865, 107.06793288])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
          